In [1]:
from bs4 import BeautifulSoup
import requests
import time
import csv

In [2]:
my_url = "https://defenders.org/wildlife"

hdr = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'}

def get_urls_from_single_page(animal_url):

    page = requests.get(animal_url, headers=hdr)
    soup = BeautifulSoup(page.text, 'html.parser')

    link_list = []

    animals = soup.find_all('div', class_='card card-linked has-image')

    for animal in animals:
        try:
            a = animal.find_all('a', class_='stretched-link')
            link_list.append(a[0].attrs['href'])
        except:
            pass

    return link_list

url_list = get_urls_from_single_page(my_url)

#print(url_list)

['/wildlife/american-crocodile-and-alligator', '/wildlife/amphibians', '/wildlife/bald-eagle', '/wildlife/bats', '/wildlife/beaver', '/wildlife/beluga-whale', '/wildlife/bighorn-sheep', '/wildlife/bison', '/wildlife/black-bear', '/wildlife/black-footed-ferret', '/wildlife/bog-turtle', '/wildlife/butterflies', '/wildlife/california-condor', '/wildlife/canada-lynx', '/wildlife/caribou', '/wildlife/dolphins', '/wildlife/elephants', '/wildlife/fisher', '/wildlife/florida-manatee', '/wildlife/florida-panther', '/wildlife/foxes', '/wildlife/freshwater-fishes', '/wildlife/freshwater-mussels', '/wildlife/golden-cheeked-warbler', '/wildlife/gray-wolf', '/wildlife/grizzly-bear', '/wildlife/hellbender', '/wildlife/horseshoe-crab', '/wildlife/jaguar', '/wildlife/lesser-prairie-chicken', '/wildlife/marbled-murrelet', '/wildlife/mexican-gray-wolf', '/wildlife/mice-and-rats', '/wildlife/north-atlantic-right-whale', '/wildlife/ocelot', '/wildlife/orca', '/wildlife/owls', '/wildlife/pacific-salmon', '/

In [3]:
gator_url = "/wildlife/mexican-gray-wolf"

def scrape_one_animal(gator_url):
    page = requests.get('https://defenders.org'+ gator_url, headers=hdr)
    soup = BeautifulSoup(page.text, 'html.parser')

    status_list = []
    

    try:
        status_container = soup.find('table', class_='table protection-table')
        status_text = status_container.find_all('span', class_='font-weight-bold small')
        stat_text = status_text[0].text
        
        if len(status_text) == 3:
            for status in status_text:
                status_list.append(status.text)
        elif len(status_text) == 2 and 'Appendix' not in status_text[1].text:
            status_list.append(status_text[0].text)
            status_list.append(status_text[1].text)
            status_list.append('Not listed')
        elif len(status_text) == 2 and 'Appendix' in status_text[1].text:
            status_list.append(status_text[0].text)
            status_list.append('Not listed')
            status_list.append(status_text[1].text)
        elif len(status_text) == 1 and 'Appendix' in stat_text:
            status_list.append('Not listed')
            status_list.append('Not listed')
            status_list.append(status_text[0].text)
        elif len(status_text) == 1 and 'Appendix' not in stat_text:
            for status in status_text:
                status_list.append(status.text)
            status_list.append('Not listed')
            status_list.append('Not listed')
        elif len(status_text) == 0:
            status_list.append('Not listed')
            status_list.append('Not listed')
            status_list.append('Not listed')

    except:
        status_list.append('Not listed')
        status_list.append('Not listed')
        status_list.append('Not listed')
    
    try:
        pop_container = soup.find('div', id='Range--Population')
        pop = pop_container.find_all('div', class_='long-text')
        pop1 = pop[1].text
        status_list.append(pop1[12:-2])
    except:
        status_list.append('Not listed')
    
    return status_list

animal_details = scrape_one_animal(gator_url)

print(animal_details)

['Endangered', 'Not listed', 'Not listed', 'In 2019 there were 163 individual wolves in 42 packs of two or more animals in the wild in Arizona and New Mexico']


In [4]:
# write a function (ONE function) to send all the actor URLs through the scrape_one_actor() function above 
# and in the same function, here, write out the following to a CSV file:
# [ actor's name, birthday, death date, partial URL ]
# in top line of the CSV, write the following column headings: actor, birthday, death, url

# do not change variable name, url_list
def write_csv(url_list):
    # open new file for writing - name it actors.csv 
    csvfile = open('animals.csv', 'w', newline='')
    
    # make a Python CSV writer object -
    c = csv.writer(csvfile)
    
    # write the column headings row 
    c.writerow( ['Endangered Species Act','IUCN Red List','CITES', 'Population', 'URL'] )

    
    # loop through the urls to run the scrape_one_actor() function for EACH url 
    # insert a 1-second time delay inside the loop 
    # write one new row into the CSV
    for url in url_list:
        animal_details = scrape_one_animal(url)
        actor = [animal_details[0], animal_details[1], animal_details[2], animal_details[3], url]
        c.writerow(actor)
        time.sleep(1)
    
    # close the file - end of function 
    csvfile.close()
    
    # function does not need a return - but the next line prevents an error in Jupyter, so keep it 
    return None

# run the function - this should create a .csv file 
# do not change variable name, url_list
write_csv(url_list)
